In [3]:
import os
from pathlib import Path

# Rutas YA listas (sin .zip)
DATA_ROOT = "/kaggle/input/data-final/Data"
DATA_YAML = os.path.join(DATA_ROOT, "data.yaml")
YOLO_ROOT = "/kaggle/working/Yolo_Experiments"  # ← AQUÍ DONDE SE GUARDAN LOS RESULTADOS

# Crear carpeta de experimentos
os.makedirs(YOLO_ROOT, exist_ok=True)

print("📁 Dataset montado en:", DATA_ROOT)
print("✅ data.yaml existe:", os.path.exists(DATA_YAML))
print("📂 YOLO_ROOT creado en:", YOLO_ROOT)

# Verificar estructura
print("\n📂 Contenido dataset:")
for item in os.listdir(DATA_ROOT):
    item_path = os.path.join(DATA_ROOT, item)
    if os.path.isdir(item_path):
        img_count = len(os.listdir(os.path.join(item_path, "images")))
        print(f"  📁 {item}/ ({img_count} imágenes)")
    else:
        print(f"  📄 {item}")


📁 Dataset montado en: /kaggle/input/data-final/Data
✅ data.yaml existe: True
📂 YOLO_ROOT creado en: /kaggle/working/Yolo_Experiments

📂 Contenido dataset:
  📄 data.yaml
  📁 valid/ (150 imágenes)
  📁 test/ (134 imágenes)
  📁 train/ (7771 imágenes)


In [2]:
!pip install ultralytics optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00a 0:00:01


In [4]:
from ultralytics import YOLO
import torch
import os
import optuna
import json

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", DEVICE)


Device: cuda


In [11]:
# Ver qué dice tu data.yaml actual
with open("/kaggle/input/data-final/Data/data.yaml", "r") as f:
    content = f.read()
    print("CONTENIDO ACTUAL:")
    print(content)


CONTENIDO ACTUAL:
path: .

train: train/images
val: valid/images
test: test/images

nc: 4
names: ['door', 'window', 'balcony', 'railing']



In [12]:
# Crear data.yaml con path ABSOLUTO + rutas relativas
DATA_ROOT = "/kaggle/input/data-final/Data"
DATA_YAML_FIXED = "/kaggle/working/data.yaml"

data_yaml_content = f"""path: {DATA_ROOT}
train: train/images
val: valid/images
test: test/images

nc: 4
names: ['door', 'window', 'balcony', 'railing']
"""

with open(DATA_YAML_FIXED, 'w') as f:
    f.write(data_yaml_content)

print("✅ data.yaml CORREGIDO:")
print(data_yaml_content)
print(f"\n📁 Guardado en: {DATA_YAML_FIXED}")


✅ data.yaml CORREGIDO:
path: /kaggle/input/data-final/Data
train: train/images
val: valid/images
test: test/images

nc: 4
names: ['door', 'window', 'balcony', 'railing']


📁 Guardado en: /kaggle/working/data.yaml


In [13]:
DATA_YAML = "/kaggle/working/data.yaml" 

def objective(trial):
    # Hyperparametros para optuna
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW"])

    # Modelo
    model = YOLO("yolov8l.pt")

    model.train(
        data=DATA_YAML,
        epochs=15, 
        patience=3,  # para si no mejora mAP durante 3 épocas consecutivas
        imgsz=512,
        batch=-1,  # YOLOv8 calcula AUTOMÁTICAMENTE el máximo batch_size que cabe en tu GPU
        lr0=lr,
        optimizer=optimizer_name,
        freeze=10,      # congelar primeras capas (backbone)
        device=DEVICE,  # cuda
        project=YOLO_ROOT,
        name=f"trial_{trial.number}",
        save=True,      # guardar best.pt / last.pt
        exist_ok=True,  # no sobrescribir conflictos
        cache=False,
        verbose=True
    )

    # Evaluación final
    val_results = model.val()

    return 1 - float(val_results.box.map50)


In [ ]:
# Crear estudio 
study = optuna.create_study(direction="minimize")

# Ejecutar Optuna
study.optimize(objective, n_trials=10)

print("Mejor configuración encontrada:")
print(study.best_params)


[I 2026-01-24 00:48:35,537] A new study created in memory with name: no-name-fdee3b6b-b931-4659-b052-2784123a88d4


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0012573172601946477, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_0, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/kaggle/working/Y

[I 2026-01-24 01:34:18,814] Trial 0 finished with value: 0.22574364879526576 and parameters: {'lr': 0.0012573172601946477, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.22574364879526576.


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.004213698863907592, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_1, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/kaggle/working/Yol

[I 2026-01-24 02:05:21,134] Trial 1 finished with value: 0.2590349217788619 and parameters: {'lr': 0.004213698863907592, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.22574364879526576.


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00018102182228359043, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_2, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/kaggle/working/

In [ ]:
import shutil
shutil.make_archive("/kaggle/working/EMERGENCY_BACKUP", 'zip', "/kaggle/working/Yolo_Experiments")
print("✅ BACKUP CREADO → Panel Output!")


In [ ]:
# RESUMEN FINAL Optuna
print("🏆 RESULTADOS FINALES Optuna:")
print(f"Total trials: {len(study.trials)}")
print(f"Mejor trial: {study.best_trial.number}")
print(f"Mejor params: {study.best_params}")
print(f"Mejor mAP50: {1 - study.best_value:.3f}")

In [ ]:
# Cargar el mejor trial
best_trial_number = study.best_trial.number
best_trial_dir = os.path.join(YOLO_ROOT, f"trial_{best_trial_number}")
best_model_path = os.path.join(best_trial_dir, "weights", "best.pt")

# Cargar modelo YOLO
from ultralytics import YOLO
best_model = YOLO(best_model_path)

# Evaluar validación final
results = best_model.val(data=DATA_YAML)

print(f"\n VALIDACIÓN FINAL:")
print(f"mAP50:     {results.box.map50:.3f}")
print(f"mAP50-95:  {results.box.map:.3f}")
print(f"Precision: {results.box.mp:.3f}")
print(f"Recall:    {results.box.mr:.3f}")

In [ ]:
# BACKUP COMPLETO de los 3 trials
!zip -r /kaggle/working/Yolo_Experiments_3trials.zip /kaggle/working/Yolo_Experiments/

echo "✅ BACKUP CREADO!"
echo "📥 Panel derecho → Output → Yolo_Experiments_3trials.zip → DOWNLOAD"
